# [Base Model Selection](https://learn.microsoft.com/azure/ai-foundry/concepts/evaluation-approach-gen-ai#base-model-selection)

The first stage of the AI lifecycle involves selecting an appropriate base model. Generative AI models vary widely in terms of capabilities, strengths, and limitations, so it's essential to identify which model best suits your specific use case. During base model evaluation, you "shop around" to compare different models by testing their outputs against a set of criteria relevant to your application.

You have three options to evaluate models:

1. [Use Azure AI Foundry Benchmarks](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/model-benchmarks) to compare models on their intrinsic capabilities.
1. [Use Manual Evaluations in the Portal](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-prompts-playground) to run prompts on models and rate them.
1. [Evaluate Multiple Models using the SDK](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-prompts-playground) code-first

**In this notebook, we explore a simplified version of option 3**

---


## Objective

You are beginning your AI application development journey - and you have two (or more) model options available to you. How do you pick the right one for your needs? In this tutorial we look at how you can evaluate _the same set of prompts_ against multiple model endpoints deployed in your Azure AI project.

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

### Validate Required Environment Variables are set

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os

assert os.environ.get("AZURE_OPENAI_ENDPOINT") is not None, "Please set the AZURE_OPENAI_ENDPOINT environment variable."
assert os.environ.get("AZURE_OPENAI_API_VERSION") is not None, "Please set the AZURE_OPENAI_API_VERSION environment variable."
assert os.environ.get("AZURE_OPENAI_API_KEY") is not None, "Please set the AZURE_OPENAI_API_KEY environment variable."
assert os.environ.get("AZURE_AI_CONNECTION_STRING") is not None, "Please set the AZURE_AI_CONNECTION_STRING environment variable."
assert os.environ.get("LAB_JUDGE_MODEL") is not None, "Please set the LAB_JUDGE_MODEL environment variable."

## Model Evaluation

We will use Evaluate API provided by Azure AI Evaluation SDK. In the notebook, we will use different models and evaluate them. Azure AI Foundry will be used to visualize and compare results


#### Getting Azure AI Foundry Project details

In [ ]:
import os

# Project Connection String
connection_string = os.environ.get("AZURE_AI_CONNECTION_STRING")

# Extract details
region_id, subscription_id, resource_group_name, project_name = connection_string.split(";")

# Populate it
azure_ai_project = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}
print(azure_ai_project)

## Data

Following code reads Json file "data.jsonl" which queries that will passed to each model for evaluation. 

In [ ]:
import pandas as pd

df = pd.read_json("00-data/05-data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [ ]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

# Note: We are evaluating 2 models above - and we need a "LLM Judge" to evaluate them
#       Here we specify the judge model
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("LAB_JUDGE_MODEL"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)
print(model_config)

## Run the evaluation

The Following code runs Evaluate API and Relevance, Coherence (LLM as Judge), Bleu, Rogue (NLP) and Violence (Content Safety) Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which calls the model.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

### Initialize the evaluators  

In [ ]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator, CoherenceEvaluator, BleuScoreEvaluator, RougeScoreEvaluator, RougeType, ViolenceEvaluator,
)
from azure.identity import DefaultAzureCredential

# LLM as judge evaluator
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)

# NLP evaluators
blue_score_evaluator = BleuScoreEvaluator()
rouge_score_evaluator = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)

# Violence evaluator
violence_evaluator = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

# Define the models to evaluate


path = str(pathlib.Path(pathlib.Path.cwd())) + "/00-data/04-data.jsonl"
print(path)

### Function that uses `evaluate` API from `azure-ai-evaluation` package to Evaluate a Base Model

In [ ]:
from model_endpoints import BaseModel

def evaluate_model(model):
    results = evaluate(
        evaluation_name=f"Base Model Evaluation {model}",
        data=path,
        target=BaseModel(model),
        evaluators={
            "relevance": relevance_evaluator,
            "coherence": coherence_evaluator,
            "blue_score": blue_score_evaluator,
            "rouge_score": rouge_score_evaluator,
            "violence_score": violence_evaluator,
        },
        azure_ai_project=azure_ai_project,
    )
    return results
    

### Evaluate GPT 35 Turbo

In [ ]:
gpt_35_turbo_results = evaluate_model("gpt-35-turbo")
pd.DataFrame(gpt_35_turbo_results["rows"])

### AI Foundry URL to view results

In [ ]:
f"AI Foundry Studio URL: {gpt_35_turbo_results['studio_url']}"

### Evaluate GPT 4o Mini

In [ ]:
gpt_4o_mini_results = evaluate_model("gpt-4o-mini")
pd.DataFrame(gpt_4o_mini_results["rows"])

## View Results In Portal

You can now visit the Azure AI Foundry portal.

1. Click the Evaluations tab to see the results from the 2 runs. 
1. You should see something like this. **Select the two runs as shown, so we can do a visual comparison of the metrics**

![base](./../docs/img/screenshots/lab-04-base-model-evals.png)

## Compare Metrics In Dashboard

1. At the top right of that panel, you will see a `Switch to Dashboard View` option. Click it!
1. In the resulting page, **collapse** the `Comparison` tab - this will `Charts` into view
1. You should see something like this - use the color cues to visually compare metrics

**For example, the chart shows that the relevance scores (chart 1) are marginally better with gpt-4o-mini (blue) for this small test dataset**

![](./../docs/img/screenshots/lab-04-base-models-dashboard.png)